In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import GRU, Dense
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [5]:
# Load the data into a Pandas dataframe
data = pd.read_csv('logon.csv')

# Convert the dataframe to a NumPy array

# Convert the date column to a pandas datetime object
# data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y %H:%M:%S')
data['date'] = pd.to_datetime(data['date'], format='%m/%d/%Y %H:%M:%S')
data['date'] = data['date'].apply(lambda x: x.timestamp())
# Encode the 'user' column as a categorical variable using one-hot encoding
le = LabelEncoder()
data['user'] = le.fit_transform(data['user'])
data['pc'] = le.fit_transform(data['pc'])
data['activity'] = le.fit_transform(data['activity'])
# Scale the data to the range of 0 to 1
dataset = data[['date', 'user', 'pc', 'activity']].to_numpy()
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# Split the data into training and testing sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train_data = dataset[0:train_size,:]
test_data = dataset[train_size:len(dataset),:]

In [6]:
def create_dataset(dataset, look_back=1):
    data_X, data_Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        data_X.append(a)
        data_Y.append(dataset[i + look_back, 0])
    return np.array(data_X), np.array(data_Y)

In [7]:
look_back = 10
train_X, train_Y = create_dataset(train_data, look_back)
test_X, test_Y = create_dataset(test_data, look_back)

# Reshape the input data to be 3-dimensional [samples, time steps, features]
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 1))

In [4]:
# Define the GRU network
model = Sequential()
model.add(GRU(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
# model.fit(train_X, train_Y, epochs=100, batch_size=1, verbose=2)

In [8]:
# Train the model
model.fit(train_X, train_Y, epochs=100, batch_size=1, verbose=2)

Epoch 1/100


In [ ]:
# Make predictions on the test data
test_predictions = model.predict(test_X)

# Rescale the data back to the original range
test_predictions = scaler.inverse_transform(test_predictions)
test_Y = scaler.inverse_transform([test_Y])

# Plot the predicted and actual values
plt.plot(test_Y[0])
plt.plot(test_predictions[:,0])
plt.show()